In [2]:
import requests
import json
import shapely
import copy
import numpy as np
from shapely.geometry import Point, Polygon
from iteration_utilities import deepflatten

In [7]:
def get(name, url="http://51.250.19.71:3400/mmodel_attrs/"):
    r = requests.get(url+name)
    return json.dumps(r.json())

In [8]:

z_masks = {"low": [[[107500.0, 69699.9921875, 0.0], [127400.0, 69699.9921875, 0.0], [127400.00000000001, 67500.0, 0.0], [129750.0, 67500.000000000044, 0.0], [129750.00000000025, 8096.0000000002547, 0.0], [9699.9999999998727, 8096.0000000005293, 0.0], [9699.9999999998763, 67500.0, 0.0], [107500.0, 67500.0, 0.0], [107500.0, 69699.9921875, 0.0]], [[9350.0000110681849, -2216.2431640625, -1173.8263247941868], [133376.25, -2216.2431640625, -1173.8263247941868], [133376.25, 70976.2421875, -1173.8263247941868], [9350.0000110681849, 70976.2421875, -1173.8263247941868], [9350.0000110681558, 67500.0, -1173.8263247941868], [107500.0, 67500.0, -1173.8263247941868], [107500.0, 69699.9921875, -1173.8263247941868], [127400.0, 69699.9921875, -1173.8263247941868], [127400.00000000001, 67500.0, -1173.8263247941868], [129750.00000000001, 67500.0, -1173.8263247941868], [129750.00000000025, 8096.0000000002547, -1173.8263247941868], [9350.000011068154, 8095.9999999996835, -1173.8263247941868], [9350.0000110681558, -2216.2431640625, -1173.8263247941868]], [[9350.0000110681849, 70976.2421875, -3052.0407116837559], [4001.1474609375, 70976.2421875, -3052.0407116837559], [433.75689697265625, 67408.8515625, -3052.0407116837559], [433.75689697265625, -2216.2431640625, -3052.0407116837559], [9350.0000110681849, -2216.2431640625, -3052.0407116837559], [9350.0000110681558, 70976.242187500015, -3052.0407116837559]]], 
           "high": [[[20963.137502540601, 2250.7827158006799, 7251.6794434684307], [45568.115314048358, 2250.7827158006799, 7251.6794434684307], [45568.115314048358, 17318.788282092599, 7251.6794434684307], [20963.137502540601, 17318.788282092599, 7251.6794434684307], [20963.137502540601, 2250.7827158006799, 7251.6794434684307]], [[45670.577565765998, 2250.7827158006799, 8319.6256510645217], [86363.137502540601, 2250.7827158006799, 8319.6256510645217], [86363.137502540601, 67846.1295177883, 8319.6256510645217], [45670.577565765998, 67846.1295177883, 8319.6256510645217], [45670.577565765998, 2250.7827158006799, 8319.6256510645217]], [[86970.054674929706, 51888.878092895597, 7942.7415416469776], [112463.13750254099, 51888.878092895597, 7942.7415416469776], [112463.13750254099, 67846.1295177883, 7942.7415416469776], [86970.054674929706, 67846.1295177883, 7942.7415416469776], [86970.054674929706, 51888.878092895597, 7942.7415416469776]], [[112570.577565766, 2250.7827158006799, 9249.4800081886096], [121765.86248016493, 2250.7827158006799, 9249.4800081886096], [121765.86248016493, 67846.129517788286, 9249.4800081886096], [112570.577565766, 67846.129517788286, 9249.4800081886096], [112570.577565766, 2250.7827158006799, 9249.4800081886096]], [[121870.577565766, 2250.7827158006799, 7979.7961803802209], [130163.13750254099, 2250.7827158006799, 7979.7961803802209], [130163.13750254099, 67846.1295177883, 7979.7961803802209], [121870.577565766, 67846.1295177883, 7979.7961803802209], [121870.577565766, 2250.7827158006799, 7979.7961803802209]], [[20770.577565766001, 49845.215922537398, 7059.1202476394283], [45563.137502540601, 49845.215922537398, 7059.1202476394283], [45563.137502540601, 67846.1295177883, 7059.1202476394283], [20770.577565766001, 67846.1295177883, 7059.1202476394283], [20770.577565766001, 49845.215922537398, 7059.1202476394283]], [[6677.5093252431097, 2250.7827158006799, 9224.1813825512982], [20670.054674929699, 2250.7827158006799, 9224.1813825512982], [20670.054674929699, 67846.1295177883, 9224.1813825512982], [6677.5093252431097, 67846.1295177883, 9224.1813825512982], [6677.5093252431097, 2250.7827158006799, 9224.1813825512982]], [[1270.5775657660399, 2250.7827158006799, 4998.3881421182195], [6563.1375025405896, 2250.7827158006799, 4998.3881421182195], [6563.1375025405896, 67846.1295177883, 4998.3881421182195], [1270.5775657660399, 67846.1295177883, 4998.3881421182195], [1270.5775657660399, 2250.7827158006799, 4998.3881421182195]], [[86470.5775657661, 2250.7827158006799, 7328.433605009297], [112463.13750254099, 2250.7827158006799, 7328.433605009297], [112463.13750254099, 15195.3741989834, 7328.433605009297], [86470.5775657661, 15195.3741989834, 7328.433605009297], [86470.5775657661, 2250.7827158006799, 7328.433605009297]]]}



In [9]:
def np_to_shapely_poly_zero(arr):
    arr_ = copy.deepcopy(arr)
    all_poly = list(deepflatten([arr_.values()],depth=2))
    for poly in all_poly:
        for point in poly:
            point[2]=0
    low_poly= [Polygon(a) for a in all_poly[0:len(arr_["low"])]]
    high_poly = [Polygon(b) for b in all_poly[len(arr_["low"])::]]
    return low_poly, high_poly
        
def np_to_shapely_points(arr):
    for a in arr:
        yield Point(*a)

In [10]:
roofshapes = get('roofshapes')
pointgrid=list(np_to_shapely_points(json.loads(roofshapes)['roofshapes']['pointgrid']))
low_polygons, high_polygons= np_to_shapely_poly_zero(z_masks)

In [16]:
def point_in_poly(low_poly=low_polygons, high_poly=high_polygons, pointgrid=pointgrid , z_masks=z_masks):
    points_list =[]
    z_low =[]
    z_high = []
    z_ = list(deepflatten([z_masks.values()],depth=2))
    for poly in z_:
        for point in poly:
            l=[]
            h=[]
            if z_.index(poly)< len(low_poly):
                l.append(point[2])
            else:
                h.append(point[2])
        z_low.append(l)
        z_high.append(h)
    z_low = list(deepflatten (z_low[0:len(low_poly)], depth=2))
    z_high = list(deepflatten (z_high[len(low_poly)::], depth=2))


    point_pairs = []
    for l in low_poly:
        for h in high_poly:
            for p in pointgrid:
                if p.within(l) and p.within(h):
                    p_low = [p.x, p.y, z_low[low_poly.index(l)]]
                    p_high = [p.x, p.y, z_high[high_poly.index(h)]]
                    point_pairs.append([p_low, p_high])
                else:
                    pass
    return point_pairs

In [20]:
a=point_in_poly()
print(a)

True